##### 环境测试

In [ ]:
print('hello world')

##### 依赖安装
- 安装LangChain
- 安装LangChain Community
- 安装LangChain OpenAI
- 安装OpenAI
- 安装Google Generativeai
（虚拟环境已安装langchain、langchain_community、langchain-openai、openai、google-generativeai依赖）

In [ ]:
# %pip uninstall langchain -y
# %pip uninstall openai -y

In [ ]:
# %pip install openai==0.27.8 -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
##### 设置环境变量
- 使用 os 模块设置进程环境变量
- 设置OpenAI访问密钥

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(".env")

# 手动设置环境变量
# os.environ["OPENAI_API_BASE_URL"] = "https://ai-yyds.com/v1"
# os.environ["OPENAI_API_KEY"] = "sk-XXXXXXXXXXXXXXXXXXXX0bDb"
# os.environ["MODEL_NAME"] = "test"

##### 环境变量读取

In [ ]:
import os

# Read the OPENAI_API_KEY from the environment
api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")


print("OPENAI_BASE_URL:{}".format(api_base))
print("OPENAI_KEY:{}".format(api_key))
print("MODEL_NAME:{}".format(model_name))

##### 检查依赖安装情况

In [ ]:
%pip show langchain
%pip show langchain_community
%pip show langchain-openai
%pip show openai
%pip show google-generativeai

##### 使用OpenAI SDK调用

In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    # base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
)

message = [
    {"role": "user", "content": "介绍下你自己"}
]

res = client.chat.completions.create(
    # model=os.getenv("MODEL_NAME"),
    model="gpt-4o-mini",
    messages=message,
    stream=False,
)

print(res)

##### 使用Google Gemini SDK调用

In [ ]:
import google.generativeai as genai

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content("介绍下你自己")

print(response.text)

##### 使用LangChain调用各种大型语言模型
`langchain_openai`是LangChain框架的一个集成包，专门用于与OpenAI的各种模型进行交互。它提供了一系列工具和接口，使得开发者可以方便地在LangChain应用中使用OpenAI的语言模型、聊天模型和嵌入模型。

In [ ]:
from langchain_openai import ChatOpenAI
import os

api_base_url = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")

# 初始化 OpenAI 模型
llm = ChatOpenAI(
    # base_url=api_base_url,
    api_key=api_key,
    model="gpt-4o-mini",
    temperature=0,
)

# 使用 invoke 方法调用模型
response = llm.invoke("介绍下你自己")

# 打印模型生成的文本
print(response)

##### 使用LangChain提词器模板
- 使用```PromptTemplate.from_template```创建自定义提词器模板
- 将提问上下文模块化，支持参数传入，提高提示词的可重用性和可维护性
- 让LLM给孩子起具有传入参数特色的特殊的名字

In [ ]:
# 起名大师
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import os

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

# 创建 PromptTemplate
prompt = PromptTemplate.from_template("你是一个起名大师,请模仿示例起3个{county}名字,比如男孩经常被叫做{boy},女孩经常被叫做{girl}")
# 生成提示
message = prompt.format(county="中国特色的",boy="狗蛋",girl="翠花")

print(message)

llm = ChatOpenAI(
    # openai_api_base=api_base,
    openai_api_key=api_key,
    model="gpt-4o-mini",
    # model=model_name,
    temperature=0,
)

llm.predict(message)

### 使用LangChain输出解释器
在 LangChain 中，输出解析器（Output Parsers）的作用是将语言模型（LLM）的原始输出转换为结构化的数据，以便在下游任务中使用。

LangChain 提供了多种内置的输出解析器，以适应不同的输出格式。以下是一些常用的解析器：
- PydanticOutputParser
  - 将 LLM 输出解析为 Pydantic 模型，适用于结构化的 JSON 输出。
- CommaSeparatedListOutputParser
  - 将 LLM 输出解析为逗号分隔的列表。
- StructuredOutputParser
  - 将LLM的输出解析为结构化的JSON格式。
- XMLOutputParser
  - 将LLM的输出解析为XML格式。

在使用某些解析器（如 PydanticOutputParser）之前，你需要定义输出数据的结构。可以使用 Pydantic 模型来定义结构。

#### 将 LLM 输出解析为逗号分隔的列表（CommaSeparatedListOutputParser）

In [ ]:
# 使用langchain内置的输出解析器
from langchain_core.output_parsers import CommaSeparatedListOutputParser

CommaSeparatedListOutputParser().parse("hi, bye")

In [ ]:
# 这是我们自己实现了将文本解析为逗号分割的列表
from langchain_core.output_parsers import BaseOutputParser

# 自定义class，继承输出解析器（Output Parsers）的基类
# 它的主要作用是定义了输出解析器的通用接口，使得 LangChain 可以统一处理不同类型的输出解析器。
class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

# 使用自定义类，格式化输出文本
CommaSeparatedListOutputParser().parse("hi, bye")

In [ ]:
# 起名大师，以数组格式输出
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import BaseOutputParser
import os

# 自定义class，继承了BaseOutputParser
class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

llm = ChatOpenAI(
    openai_api_base=api_base,
    openai_api_key=api_key,
    model="gpt-4o-mini",
    temperature=0,    
)

# 创建提示模板
prompt = PromptTemplate.from_template("你是一个起名大师,请模仿示例起4个{county}名字,比如男孩经常被叫做{boy},女孩经常被叫做{girl}")
message = prompt.format(county="中国特色的",boy="狗蛋",girl="翠花")

strs = llm.invoke(message)

# 创建输出解析器并解析LLM的输出结果
CommaSeparatedListOutputParser().parse(strs)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# 定义 Pydantic 模型
class Joke(BaseModel):
    setup: str = Field(description="笑话设置")
    punchline: str = Field(description="笑话的妙语")

# 创建输出解析器
parser = PydanticOutputParser(pydantic_object=Joke)
# 打印为语言模型（LLM）提供格式化输出的指令
# parser.get_format_instructions() 方法会返回一个字符串，其中包含有关如何格式化 LLM 输出的指令，以便解析器可以正确地解析它。这些指令通常包括：
# 输出格式的描述、字段或属性的描述
print(parser.get_format_instructions())

# 创建提示模板
template = """给我讲个笑话。

{format_instructions}"""

prompt = PromptTemplate(
    template=template,
    input_variables=[],
    # 输出格式说明
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 创建 LLM 链
chain = prompt | OpenAI(temperature=0) | parser

# 调用 LLM 链
output = chain.invoke({})

print(output)